# Tool Overview

This tool should allow you to read in a csv file with latitude, longitude, and year columns of point data (specifically designed for forest plots) and output a csv file and a feature class that both contain values for the palmer drought severity index in the county of the point in the previous summer, the total popultation in the appropriate year and the population/square mile of the county that the point falls into. All data besides the CSV are accessed via API's. However, this tool will only work when the CSV data has a year range between 2000 and 2018, otherwise I will need to make additional requests on the census API. This tool has 8 inputs:

0. space --> where we will create a geodatabase to store output and store our ouput CSV
1. incsv --> input csv with lat lon and year component
2. longitude --> (will be set to lon in tool) the column in the csv where longitude is stored
3. latitude --> (will be set to lat in tool) the column in the csv where latitude is stored
4. plot_year --> (will be set to invyr in tool) the column in the csv where the year is stored
5. usename --> username for arcgis (I give permission to use my username: hjs5td)
6. passwrd --> password for arcgis (password: Hondoo12)
7. key --> key for Census API (I give permission to use my key: a8240e46395d6100bb5585e555c6c5be99584107)

as space and the incsv to match your computer, this should work.



# Section 1 : Reading a CSV and Create Point FeatureClass of Plot Data

## Step 1.
**Import Python Modules**
    - arc modules
        - arcgis gives access to arcgis api
        - arcpy gives access to eveything available in arcpro software (functions, layout etc.)
    - other moduels
        - pandas is used for data analysis and manipulation
        - numpy is used for math
        - sodapy is a module recommended by CDC to use their API
        - zipfile used to unzip zipfiles
        - os gives access to basic functionalities of the computer (creating folders, path names, etc.)

In [1]:
import arcgis
import arcpy
import pandas as pd
import numpy as np
import sodapy
from zipfile import ZipFile
import os
import json
import requests


## Step 2.
**set up inputs to convert CSV to featureclass**

In [2]:
# set up directory and geodatabase

space = os.getcwd()

incsv = os.path.join('in_data','plot_sample.csv')

# these are the names of longitude and latitude in your CSV 

longitude = 'lon'

latitude = 'lat'

plot_year = "invyr"

## Step 3.
**create a geodatabase where we will store our data**

In [3]:
outgdb = "AdvGIS_proj.gdb"
workspace = os.path.join(space, outgdb)

if arcpy.Exists(workspace):
    arcpy.Delete_management(workspace)

arcpy.CreateFileGDB_management(space,outgdb)

arcpy.env.workspace = workspace
arcpy.env.overwriteOutput = True

## Step 4.
**create a featureclass from CSV**

In [4]:
#define name of featureclass to go into geodatabase
plots = "plots"
plottab = "plot_table"

#GCS_WGS_1984 geographic coordinate system
sr = arcpy.SpatialReference(4326)

#USA_Contiguous_Albers_Equal_Area_Conic_USGS_version projected coordinate system
pr = arcpy.SpatialReference(102039) 

arcpy.XYTableToPoint_management(incsv,plots, x_field = longitude, y_field = latitude, coordinate_system = sr)


<Result 'C:\\Users\\hjs5td\\Desktop\\SudekumFinalProject\\AdvGIS_proj.gdb\\plots'>

## Step 5.
**create new feature class projected into equal area projection**

In [5]:
plots_proj = 'plots_proj'

transformation = 'WGS_1984_(ITRF00)_To_NAD_1983'

#going from sr to pr coordinate system

arcpy.Project_management(plots, plots_proj, pr, transformation, sr)

<Result 'C:\\Users\\hjs5td\\Desktop\\SudekumFinalProject\\AdvGIS_proj.gdb\\plots_proj'>

# Section 2 : Accessing APIS and Getting Explanatory Data

# Download a Layer From ArcGIS hub
http://hub.arcgis.com/pages/open-data

## Step 1.
**Set up text inputs for accessing various API's**

In [6]:
# inputs:

# username and password of arcgis account

usename = ''
passwrd = ''

# census key can be requested at : https://api.census.gov/data/key_signup.html 

key = ''


## Step 2. 
**set up numeric variables to be used for PDSI API**

In [7]:
# used in the sql clause for CDC API
# does not require a key or username+pass

minmonth = 6
maxmonth = 8
minyear = 1998

## Step 3.
**get URL of a county feature layer from arcgis hub website.**

In [8]:
URL = "https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Counties_Generalized/FeatureServer/0/"
county_layer = arcgis.features.FeatureLayer(URL)
county_layer

<FeatureLayer url:"https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Counties_Generalized/FeatureServer/0/">

**you can use the code below to see what is allowed with this dataset; it says "extract" you can directly download the layer as shapefile. see https://developers.arcgis.com/python/guide/checking-out-data-from-feature-layers-using-replicas/**

In [9]:
county_layer.properties.capabilities

'Query'

## Step 4. (not necessary for the tool)
**create a GIS() object; plot a map using the created GIS() object.** Need to create an account in order to download layers directly -- (my username and password are provided in the tool for simplicity)

In [10]:
gis1 = arcgis.gis.GIS(username=usename, password=passwrd)
map1 = gis1.map("Missouri")
map1

MapView(layout=Layout(height='400px', width='100%'))

In [11]:
## add to a map from a url

map1.add_layer({"type":"FeatureLayer",
                "url":URL,
               })

## Step 5.
**query all features of the layer.**

In [12]:
all_features = county_layer.query()

print('Total number of rows in the dataset: ')
print(len(all_features.features))

Total number of rows in the dataset: 
3142


## Step 6. 
**create spatial dataframe from the county_layer.**

In [13]:
# store as dataframe pandas dataframe
# this works if we can extract the features

sdf = pd.DataFrame.spatial.from_layer(county_layer)

## Step 7. 
**delete the columns that we do not need**

In [14]:
keeperlist = ['FID', 'FIPS', 'OBJECTID', 'SHAPE', 'SQMI', 'STATE_FIPS', 'STATE_NAME', 'Shape_Area', 'Shape_Leng', 
              'Shape__Area', 'Shape__Length']

# FIPS = sdf.FIPS
# print(FIPS.head())

for i in list(sdf):
    if not i in keeperlist:
        del sdf[i]

print(list(sdf))     

['FID', 'OBJECTID', 'STATE_NAME', 'STATE_FIPS', 'FIPS', 'SQMI', 'Shape_Leng', 'Shape_Area', 'Shape__Area', 'Shape__Length', 'SHAPE']


# Get and Format Drought Data
https://open.cdc.gov/apis.html

## Step 8.
**create a string that can take inputs of minimum year, minimum month and maximum month.**


In [15]:
# from our inputs ... this will control which months the drought data is gathered from (1-12 would get all months)
# I have set it to 6-8 because we have found that june through august is most important for forest conditions
# later on the palmer drought severity index values are averaged over these months

monthlist = list(range(minmonth,maxmonth+1))
final = str()
length = len(monthlist) - 1

for idx,i in enumerate(monthlist):
    if idx < length:
        string1 = "'{}',".format(i)
        final = final + string1
    else:
        string1 = "'{}'".format(i)
        final = final + string1
    
monthstring = " AND month IN ({})".format(final)

# minimum value for this clause is 1894
clause = "year > '{}'".format(minyear) + monthstring

print(clause)

year > '1998' AND month IN ('6','7','8')


## Step 9. 
**Using sodapy library (recommended by CDC) get the drought data from CDC API.** set limit to something large so we get all records (if excluded will only get 1000).

In [16]:
# this gets the pdsi data from cdc website using the query speciried in the get() function

client = sodapy.Socrata("data.cdc.gov", None)
results = client.get("en5r-5ds4", where=clause, limit = 10000000)
pdsi = pd.DataFrame.from_records(results)

client.close()

In [17]:
print(pdsi.head(5))
print(len(pdsi))

   year month statefips countyfips   pdsi
0  1999     6         1       1001   1.24
1  1999     6         1       1003  -1.51
2  1999     6         1       1005   1.41
3  1999     6         1       1007   1.24
4  1999     6         1       1009   1.81
167886


## Step 10.
**change datatypes of all columns.**

In [18]:
# make sure all countyfips are stored as integers
# countyfips is used to join to the layer
# these are originally stored as text

intlist = ["countyfips","year","month","statefips"]

for i in list(pdsi):
    if i in intlist:
        pdsi[str(i)] = pd.to_numeric(pdsi[str(i)],downcast='integer')
    else:
        pdsi[str(i)] = pd.to_numeric(pdsi[str(i)],downcast='float')

    

## Step 11.
**iterate through yearlist and store dataframes in dictionary.** merge all dataframes to one dataframe called all_drought.

In [19]:

# get the maximum value from pdsi["year"] column
maxyear = max(pdsi["year"])

# min year plus one because we used > min year not => minyear
# we will iterate through this list and use values as keys/column names

yearlist = list(range(minyear+1,maxyear+1))

# create dictionary to easily store dataframes as we iterate
D = {}

# enumerate the list so we can know when we are on the first iteration (see below)

for idx,i in enumerate(yearlist):
    # get values where year == i
    D[i] = pdsi[pdsi.year == i]
    # monthlist defined in step 8
    D[i] = D[i][D[i].month.isin(monthlist)]
    # groupby county and find the average pdsi across the months in our list
    D[i] = D[i].groupby(['countyfips'])[['pdsi']].mean().reset_index()
    # get a string of the value in yearlist
    yearsuf = str(i)
    # create a column name
    pdsiname = "pdsi{}".format(yearsuf)
    # reassign the column name to the PDSI column
    D[i] = D[i].rename(columns={'pdsi': pdsiname})
    # if it is the first iteration than store the dataframe as all_drought
    if idx == 0:
        all_drought = D[i]
    # otherwise merge the dataframes on the county number
    else:
        all_drought = all_drought.merge(D[i], how='inner', on='countyfips')
    # print(len(all_drought))

del D

## Step 12.
**Make sure that FIPS in the spatial dataframe is stored as an integer. Then merge the drought data to the county data using the countyfips and FIPS keys.**

In [20]:
sdf['FIPS'] = pd.to_numeric(sdf['FIPS'],downcast='integer')

In [21]:
pdsicounty = sdf.merge(all_drought, how='inner', left_on='FIPS', right_on='countyfips')

In [22]:
print(pdsicounty.head())

   FID  OBJECTID    STATE_NAME STATE_FIPS   FIPS    SQMI  Shape_Leng  \
0   19      1215  South Dakota         46  46029  717.30    1.933182   
1   22      1218  South Dakota         46  46039  636.68    1.730427   
2   27      1223  South Dakota         46  46057  538.03    1.741057   
3   56      1252       Alabama         01   1011  625.05    2.006250   
4   57      1253       Alabama         01   1013  777.88    1.769462   

   Shape_Area   Shape__Area  Shape__Length  ...  pdsi2007  pdsi2008  pdsi2009  \
0    0.216860  3.799034e+09  247886.441856  ...  3.336667  2.440000  4.470000   
1    0.186499  3.254596e+09  231916.025980  ...  3.243333  2.813334  4.300000   
2    0.158622  2.764034e+09  217296.620249  ...  3.243333  2.813334  4.300000   
3    0.155727  2.278179e+09  241042.982881  ... -4.353333 -1.440000  1.216667   
4    0.192731  2.808881e+09  213677.638219  ... -3.636667 -1.366667  1.493333   

   pdsi2010  pdsi2011  pdsi2012  pdsi2013  pdsi2014  pdsi2015  pdsi2016  
0  6.9

In [23]:
print(list(pdsicounty))

['FID', 'OBJECTID', 'STATE_NAME', 'STATE_FIPS', 'FIPS', 'SQMI', 'Shape_Leng', 'Shape_Area', 'Shape__Area', 'Shape__Length', 'SHAPE', 'countyfips', 'pdsi1999', 'pdsi2000', 'pdsi2001', 'pdsi2002', 'pdsi2003', 'pdsi2004', 'pdsi2005', 'pdsi2006', 'pdsi2007', 'pdsi2008', 'pdsi2009', 'pdsi2010', 'pdsi2011', 'pdsi2012', 'pdsi2013', 'pdsi2014', 'pdsi2015', 'pdsi2016']


## Step 13.
**write our merged spatial dataframe to a feature class.**  store it in the geodatabase.

In [24]:
pdsipath = os.path.join(workspace,"pdsi")

pdsicounty.spatial.to_featureclass(location = pdsipath)

'C:\\Users\\hjs5td\\Desktop\\SudekumFinalProject\\AdvGIS_proj.gdb\\pdsi'

# Get and Download Population Data

## Step 14.
**access the census API for county level population estimates using for years 2000 to 2010**

In [25]:
## see https://api.census.gov/data/2000/pep/int_population/examples.html ##

# this gets county level population estimates from 2000 to 2010

URL = "https://api.census.gov/data/2000/pep/int_population?get=COUNTY,DATE_DESC,POP,GEONAME&for=county:*&in=state:*&key={}".format(key)

data = requests.get(URL)

data

<Response [200]>

## Step 15.
**convert json to nested list using the json() function**

In [26]:
datajson = data.json()

## Step 16.
**use pandas to write a dataframe from the datajson list**

In [27]:
cp = pd.DataFrame.from_records(datajson)

In [28]:
print(cp.head())
print(len(cp))

        0                                   1      2                        3  \
0  COUNTY                           DATE_DESC    POP                  GEONAME   
1     003  4/1/2000 population estimates base  73943  Aroostook County, Maine   
2     003        7/1/2000 population estimate  73872  Aroostook County, Maine   
3     003        7/1/2001 population estimate  72962  Aroostook County, Maine   
4     003        7/1/2002 population estimate  72942  Aroostook County, Maine   

       4       5  
0  state  county  
1     23     003  
2     23     003  
3     23     003  
4     23     003  
38653


In [29]:
list(cp)

[0, 1, 2, 3, 4, 5]

## Step 17.
**make the first row the coulumn names and drop that row**

In [30]:
cp.columns = cp.iloc[0]
cp = cp.reindex(cp.index.drop(0))

In [31]:
print(list(cp))
print(len(cp))

['COUNTY', 'DATE_DESC', 'POP', 'GEONAME', 'state', 'county']
38652


In [32]:
print(cp.head())

0 COUNTY                           DATE_DESC    POP                  GEONAME  \
1    003  4/1/2000 population estimates base  73943  Aroostook County, Maine   
2    003        7/1/2000 population estimate  73872  Aroostook County, Maine   
3    003        7/1/2001 population estimate  72962  Aroostook County, Maine   
4    003        7/1/2002 population estimate  72942  Aroostook County, Maine   
5    003        7/1/2003 population estimate  72944  Aroostook County, Maine   

0 state county  
1    23    003  
2    23    003  
3    23    003  
4    23    003  
5    23    003  


## Step 18.
**do the same thing for county level data from 2010 to 2018**

In [33]:

# if we do not include "DATE_CODE" than we only get the 2018 estimate
URL = "https://api.census.gov/data/2018/pep/population?get=COUNTY,DATE_CODE,DATE_DESC,POP,GEONAME&for=county:*&in=state:*&key={}".format(key)

data = requests.get(URL)

datajson = data.json()


In [34]:
ncp = pd.DataFrame.from_records(datajson)

ncp.columns = ncp.iloc[0]
ncp = ncp.reindex(ncp.index.drop(0))

# delete DATE_CODE
del ncp["DATE_CODE"]


## Step 19. 
**concatenate the two dataframes (stack them on top of one another); must have same columns; reset the index so that all values are unique and delete the old index column**

In [35]:
# since we redefine cp here make sure that we rerun the first request sequence before we concatanate dataframes again

cp = pd.concat([ncp,cp], axis=0, sort = False, join='outer',ignore_index=True)

In [36]:
cp = cp.reset_index()

In [37]:
del cp["index"]

## Step 20. 
**iterate through the indexes of the cp dataframe to format the FIPS and year variables.** I tried usiung datetime package but it was simpler to just format it on my own.

In [38]:
stringlist = ["population estimate", "population estimates base","Census 2010 population","Census population"]

# create an empty column year
cp["year"] = ""
#create an empty column FIPS
cp["FIPS"] = ""
#create an empty list to store year values
yearlist = list()

# iterate through the indexs
# iterating through a pandas dataframe
for i in cp.index:
    datestr = cp.at[i,"DATE_DESC"]
    county = str(cp.at[i,"county"])
    state = str(cp.at[i,"state"])
    FIPS = int(state + county)
    cp.at[i,"FIPS"] = FIPS
    # if any(x in datestr for x in stringlist):
    for y in stringlist:
        # check to see if datestr created above ends with any value in the list
        if datestr.endswith(y):
            # get the length of the text that the datestr endswith
            length = len(y)
            # create a negative integer value from that length
            position = -length
            # new string excluding that text
            new_datestr = datestr[:position]
            # new string excluding month and day
            # includes the space and the year I beleive
            new_year = new_datestr[-5:]
            # convert to integer
            year = int(new_year)
            # assign value
            cp.at[i,"year"] = year
            # to check the min and max later on
            yearlist.append(year)

# we need to drop duplicates because each county includes a baseline 2000 census estimate and an additional 2000 estimate
# I sort values so that we are always dropping the baseline estimate
            
cp = cp.sort_values(['state','county','year'], axis=0, ascending=True,kind='quicksort', na_position='first')

cp = cp.drop_duplicates(subset=['state','county','year'], keep='last', inplace=False)   

## Step 21.
**create a new list of the minimum year and the maximum years found in the dataframe**

In [39]:
yeararray = np.array(yearlist)

x = min(yeararray)
y = max(yeararray)

yearlist = list(range(x,y+1))

del yeararray
print(list(cp))

['COUNTY', 'DATE_DESC', 'POP', 'GEONAME', 'state', 'county', 'year', 'FIPS']


## Step 22.
**select rows by year; rename column; mrege dataframes (same as drought method)**

In [40]:
D = {}

for idx,i in enumerate(yearlist):
    print(i)
    D[i] = cp[cp.year == i]
    # groupby county and find the average pdsi
    yearsuf = str(i)
    name = "pop{}".format(yearsuf)
    D[i] = D[i].rename(columns={'POP' : name})
    if idx == 0:
        all_pop = D[i]
    else:
        # get a subselection of columns
        D[i] = D[i][[name, 'FIPS']]
        print(list(D[i]))
        all_pop = all_pop.merge(D[i], how='inner', on='FIPS')
    #print(len(all_pop))
    #print(idx)


2000
2001
['pop2001', 'FIPS']
2002
['pop2002', 'FIPS']
2003
['pop2003', 'FIPS']
2004
['pop2004', 'FIPS']
2005
['pop2005', 'FIPS']
2006
['pop2006', 'FIPS']
2007
['pop2007', 'FIPS']
2008
['pop2008', 'FIPS']
2009
['pop2009', 'FIPS']
2010
['pop2010', 'FIPS']
2011
['pop2011', 'FIPS']
2012
['pop2012', 'FIPS']
2013
['pop2013', 'FIPS']
2014
['pop2014', 'FIPS']
2015
['pop2015', 'FIPS']
2016
['pop2016', 'FIPS']
2017
['pop2017', 'FIPS']
2018
['pop2018', 'FIPS']


## Step 23.
**merge pop data to spatial dataframe**

In [41]:
popcounty = sdf.merge(all_pop, how='inner', on='FIPS')

## Step 24.
**write population data to geodatabse**

In [42]:
poppath = os.path.join(workspace,"pop")


popcounty.spatial.to_featureclass(location = poppath)

'C:\\Users\\hjs5td\\Desktop\\SudekumFinalProject\\AdvGIS_proj.gdb\\pop'

## Step 25.
**Spatial Joins of Point**

https://developers.arcgis.com/python/guide/spatially-enabled-dataframe-advanced-topics/#Spatial-Joins

In [43]:
arcpy.MakeFeatureLayer_management(plots_proj, 'tempplots')
arcpy.MakeFeatureLayer_management(pdsipath, 'temppdsi')
arcpy.MakeFeatureLayer_management(poppath, 'temppop')

<Result 'temppop'>

In [44]:
orig = arcpy.ListFields('tempplots')
origlist = list()


for i in orig:
    x = i.name
    origlist.append(x)

In [45]:
temp = "temporaryjoin"

arcpy.SpatialJoin_analysis('tempplots','temppop',temp, join_operation = 'JOIN_ONE_TO_ONE', join_type = 'KEEP_ALL', match_option = 'CLOSEST')


<Result 'C:\\Users\\hjs5td\\Desktop\\SudekumFinalProject\\AdvGIS_proj.gdb\\temporaryjoin'>

In [46]:
temp1 = "temporatyjoin1"

arcpy.SpatialJoin_analysis(temp,'temppdsi',temp1, join_operation = 'JOIN_ONE_TO_ONE', join_type = 'KEEP_ALL', match_option = 'CLOSEST')


<Result 'C:\\Users\\hjs5td\\Desktop\\SudekumFinalProject\\AdvGIS_proj.gdb\\temporatyjoin1'>

## Step 26.
**convert to spatial dataframe and format**

In [47]:
pltdf = pd.DataFrame.spatial.from_featureclass(temp1)

In [48]:
deletelist = ['OBJECTID',
 'Join_Count',
 'TARGET_FID',
 'Join_Count_1',
 'TARGET_FID_1',
 'Field1',
 'FID_1',
 'FIPS_1',
 'SQMI_1',
 'STATE_FIPS_1',
 'STATE_NAME_1',
 'Shape_Leng_1',
 'Shape__Area_1',
 'Shape__Length_1',
 'countyfips',
 'STATE_FIPS',
 'STATE_NAME',
 'Shape_Leng',
 'Shape__Area',
 'Shape__Length',
 'COUNTY',
 'DATE_DESC','FID']

for i in deletelist:
    if hasattr(pltdf, i):
        del pltdf[i]

In [49]:
yearlist = list(range(2000,2018))
columnlist = list(pltdf)

pltdf['final_pdsi'] = ""
pltdf['final_pop'] = ""

for i in pltdf.index:
    popstr = str(int(pltdf.at[i,plot_year]))
    # we do this because we are interested in drought of the previous year rather than drought of the current year
    pdsistr = str(int(pltdf.at[i,plot_year]) - 1)
    for y in columnlist:
        # search for columns with the name
        if popstr in y:
            if y.startswith('pop'):
                pltdf.at[i,"final_pop"] = pltdf.at[i,y]
        elif pdsistr in y:
            if y.startswith('pdsi'):
                pltdf.at[i,"final_pdsi"] = pltdf.at[i,y]

    

In [50]:
newlist = ['FIPS','SQMI','final_pdsi','final_pop']
for i in newlist:
    origlist.append(i)

In [51]:
for y in list(pltdf):
    if not y in origlist:
        del pltdf[y]

## Step 27.
**create a relative population from square miles and the correct population**

In [52]:
pltdf['final_pop'] = pd.to_numeric(pltdf['final_pop'],downcast='float')
pltdf['SQMI'] = pd.to_numeric(pltdf['SQMI'],downcast='float')

pltdf.loc[:,'poparea'] = pltdf.final_pop / pltdf.SQMI

## Step 28.
**write our results to a csv and a featureclass**

In [53]:
plotpath = os.path.join(workspace,"update_plots")
out = os.path.join(space,'out_data')
csvpath = os.path.join(out,"SudekumPlotSampleUpdate.csv")

pdsicounty.spatial.to_featureclass(location = plotpath)
pltdf.to_csv(csvpath)